### in this script, all kind of possible keys are investigated 

In [1]:
#### helper func to compare daq and phyiscs frames

def norm(x):
    return None if (isinstance(x, float) and math.isnan(x)) else x

In [2]:
from icecube import icetray, dataio, dataclasses, LeptonInjector
import math
from icecube.dataclasses import ModuleKey
from icecube.icetray import OMKey

In [3]:
key1 = OMKey(282, 17, 10)  
key2 = OMKey(339, 20, 11)
key3 = OMKey(1  , 4,  10)
key4 = OMKey(282, 17, 0)

In [4]:
DATA_PATH = '/project/def-nahee/kbas/Graphnet-Applications/Playground/SingleI3Batch/pom_response_including_every_keys.i3.gz'
data_file = dataio.I3File(DATA_PATH)

In [5]:
data_file = dataio.I3File(DATA_PATH)
frame_daq = data_file.pop_frame()
frame_physics = data_file.pop_frame()

In [6]:
Accepted_PulseMap_physics = frame_physics["Accepted_PulseMap"]
Accepted_PulseMap_daq = frame_daq["Accepted_PulseMap"]
####
PMT_Response_nonoise_physics = frame_physics["PMT_Response_nonoise"]
PMT_Response_nonoise_daq = frame_daq["PMT_Response_nonoise"]
# PMT seviyesinde ve sadece gercek pulse
####
PMT_Response_physics = frame_physics["PMT_Response"]
PMT_Response_daq = frame_daq["PMT_Response"]
# PMT seviyesinde ve noise + signal
####
EventPulseSeries_physics = frame_physics["EventPulseSeries"]
EventPulseSeries_daq = frame_daq["EventPulseSeries"]
####
EventPulseSeries_nonoise_physics = frame_physics["EventPulseSeries_nonoise"]
EventPulseSeries_nonoise_daq = frame_daq["EventPulseSeries_nonoise"]
####
Noise_K40_physics = frame_physics["Noise_K40"]
Noise_K40_daq = frame_daq["Noise_K40"]
####
Noise_Dark_physics = frame_physics["Noise_Dark"]
Noise_Dark_daq = frame_daq["Noise_Dark"]
####
triggerpulsemap_physics = frame_physics["triggerpulsemap"] 
triggerpulsemap_daq = frame_daq["triggerpulsemap"] 

In [7]:
n_daq = 0
n_physics = 0
n_other = {}

f = dataio.I3File(DATA_PATH)

while f.more():
    fr = f.pop_frame()
    stop = fr.Stop

    if stop == icetray.I3Frame.DAQ:
        n_daq += 1
    elif stop == icetray.I3Frame.Physics:
        n_physics += 1
    else:
        n_other[str(stop)] = n_other.get(str(stop), 0) + 1

f.close()

In [8]:
print("DAQ:", n_daq)
print("Physics:", n_physics)
print("Other:", n_other)

DAQ: 28
Physics: 28
Other: {}


In [9]:
print(frame_daq)

[ I3Frame  (DAQ):
  'Accepted_PulseMap' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (2508)
  'DOMTrigger_HitTimes' [DAQ] ==> I3Map<OMKey, vector<double> > (942)
  'DOMTrigger_NCoin' [DAQ] ==> I3Map<OMKey, vector<int> > (203)
  'DOMTrigger_PMTs' [DAQ] ==> I3Map<OMKey, vector<int> > (527)
  'DOMTrigger_Time' [DAQ] ==> I3Map<OMKey, vector<double> > (294)
  'DetectorTriggers_3PMT_2DOM' [DAQ] ==> I3Vector<double> (38)
  'EventProperties' [DAQ] ==> LeptonInjector::BasicEventProperties (140)
  'EventPulseSeries' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (145016)
  'EventPulseSeries_nonoise' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (1462)
  'I3EventHeader' [DAQ] ==> I3EventHeader (99)
  'I3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (422)
  'I3MCTree_RNGState' [DAQ] ==> I3GSLRandomServiceState (87)
  'I3MCTree_postprop' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (4026)
  'I3Photons' [DAQ] ==> I3Map<ModuleKey, I3Vector

In [10]:
print(frame_physics)

[ I3Frame  (Physics):
  'Accepted_PulseMap' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (2508)
  'DOMTrigger_HitTimes' [DAQ] ==> I3Map<OMKey, vector<double> > (942)
  'DOMTrigger_NCoin' [DAQ] ==> I3Map<OMKey, vector<int> > (203)
  'DOMTrigger_PMTs' [DAQ] ==> I3Map<OMKey, vector<int> > (527)
  'DOMTrigger_Time' [DAQ] ==> I3Map<OMKey, vector<double> > (294)
  'DetectorTriggers_3PMT_2DOM' [DAQ] ==> I3Vector<double> (38)
  'EventProperties' [DAQ] ==> LeptonInjector::BasicEventProperties (140)
  'EventPulseSeries' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (145016)
  'EventPulseSeries_nonoise' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (1462)
  'I3EventHeader' [DAQ] ==> I3EventHeader (99)
  'I3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (422)
  'I3MCTree_RNGState' [DAQ] ==> I3GSLRandomServiceState (87)
  'I3MCTree_postprop' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (4026)
  'I3Photons' [DAQ] ==> I3Map<ModuleKey, I3Ve

## 1- Accepted_PulseMap

In [11]:
same_nan_ok = all(
    [(p.time, p.charge, norm(p.width)) for p in Accepted_PulseMap_daq[k]] ==
    [(p.time, p.charge, norm(p.width)) for p in Accepted_PulseMap_physics[k]]
    for k in Accepted_PulseMap_physics.keys()
)


print("Accepted_PulseMap (DAQ) and Accepted_PulseMap (Physics) identical (if NaN treated equal):", same_nan_ok)


Accepted_PulseMap (DAQ) and Accepted_PulseMap (Physics) identical (if NaN treated equal): True


## 2- PMT_Response_nonoise

In [12]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in PMT_Response_nonoise_daq[k]] ==
    [(p.time, p.charge, p.width) for p in PMT_Response_nonoise_physics[k]]
    for k in PMT_Response_nonoise_physics.keys()
)


print("PMT_Response_nonoise (DAQ) and PMT_Response_nonoise (Physics) identical:", same_nan_ok)


PMT_Response_nonoise (DAQ) and PMT_Response_nonoise (Physics) identical: True


## 3- PMT_Response

In [13]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in PMT_Response_daq[k]] ==
    [(p.time, p.charge, p.width) for p in PMT_Response_physics[k]]
    for k in PMT_Response_physics.keys()
)


print("PMT_Response(DAQ) and PMT_Response (Physics) identical:", same_nan_ok)


PMT_Response(DAQ) and PMT_Response (Physics) identical: True


## 4- EventPulseSeries

In [14]:
same_nan_ok = all(
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_daq[k]] ==
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_physics[k]]
    for k in EventPulseSeries_physics.keys()
)


print("EventPulseSeries (DAQ) and EventPulseSeries (Physics) identical (if NaN treated equal):", same_nan_ok)

EventPulseSeries (DAQ) and EventPulseSeries (Physics) identical (if NaN treated equal): True


## 5- EventPulseSeries_nonoise 

In [15]:
same_nan_ok = all(
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_nonoise_daq[k]] ==
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_nonoise_physics[k]]
    for k in EventPulseSeries_nonoise_physics.keys()
)


print("EventPulseSeries_nonoise (DAQ) and EventPulseSeries_nonoise (Physics) identical (if NaN treated equal):", same_nan_ok)

EventPulseSeries_nonoise (DAQ) and EventPulseSeries_nonoise (Physics) identical (if NaN treated equal): True


## 6- Noise_K40

In [16]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in Noise_K40_daq[k]] ==
    [(p.time, p.charge, p.width) for p in Noise_K40_physics[k]]
    for k in Noise_K40_physics.keys()
)


print("Noise_K40 (DAQ) and Noise_K40 (Physics) identical:", same_nan_ok)


Noise_K40 (DAQ) and Noise_K40 (Physics) identical: True


## 7- Noise_Dark

In [17]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in Noise_Dark_daq[k]] ==
    [(p.time, p.charge, p.width) for p in Noise_Dark_physics[k]]
    for k in Noise_Dark_physics.keys()
)


print("Noise_Dark (DAQ) and Noise_Dark (Physics) identical:", same_nan_ok)


Noise_Dark (DAQ) and Noise_Dark (Physics) identical: True


In [18]:
# print("n keys:", len(Accepted_PulseMap_physics))     

In [19]:
#for k in Accepted_PulseMap_physics.keys():
#    print(k)

## 8- I3Photons

In [20]:
print(frame_daq['I3Photons'])

In [21]:
print(frame_physics['I3Photons'])

In [22]:
print(frame_daq.type_name("I3Photons"))
print(frame_physics.type_name("I3Photons"))


I3Map<ModuleKey, I3Vector<I3CompressedPhoton> >
I3Map<ModuleKey, I3Vector<I3CompressedPhoton> >


#### idk why, I3Photons do not work here. but it worked in clsim data. find out later

# 9- EventProperties


In [23]:
print(frame_physics['EventProperties'])

[ BasicEventProperties 
         TotalEnergy : 993.477
              Zenith : 2.41197
             Azimuth : 5.61741
         FinalStateX : 0.361584
         FinalStateY : 0.819704
          FinalType1 : MuMinus
          FinalType2 : Hadrons
         InitialType : NuMu
                   X : -589.976
                   Y : 399.972
                   Z : 296.257
    TotalColumnDepth : 698571
              Radius : 712.776
     ImpactParameter : 336.258
]


In [24]:
print(frame_daq['EventProperties'])

[ BasicEventProperties 
         TotalEnergy : 993.477
              Zenith : 2.41197
             Azimuth : 5.61741
         FinalStateX : 0.361584
         FinalStateY : 0.819704
          FinalType1 : MuMinus
          FinalType2 : Hadrons
         InitialType : NuMu
                   X : -589.976
                   Y : 399.972
                   Z : 296.257
    TotalColumnDepth : 698571
              Radius : 712.776
     ImpactParameter : 336.258
]


# 10- I3MCTree

In [25]:
print(frame_daq['I3MCTree'])

[I3MCTree:
  136 NuMu (-589.976m, 399.972m, 296.257m) (138.196deg, 321.854deg) 0ns 993.477GeV 0m Primary
    130 MuMinus (-589.976m, 399.972m, 296.257m) (136.385deg, 317.974deg) 0ns 179.12GeV nanm MCTrack
    133 Hadrons (-589.976m, 399.972m, 296.257m) (138.576deg, 322.743deg) 0ns 814.357GeV nanm Cascade
]


In [26]:
 print(frame_physics['I3MCTree'])

[I3MCTree:
  136 NuMu (-589.976m, 399.972m, 296.257m) (138.196deg, 321.854deg) 0ns 993.477GeV 0m Primary
    130 MuMinus (-589.976m, 399.972m, 296.257m) (136.385deg, 317.974deg) 0ns 179.12GeV nanm MCTrack
    133 Hadrons (-589.976m, 399.972m, 296.257m) (138.576deg, 322.743deg) 0ns 814.357GeV nanm Cascade
]


# 11- I3EventHeader

In [27]:
print(frame_daq['I3EventHeader'])

[ I3EventHeader:
        StartTime: 1905-06-04 18:35:44.000,000,000,0 UTC
         EndTime : 1905-06-04 18:35:44.000,000,000,0 UTC
           RunID : 4910
        SubrunID : 4294967295
         EventID : 7
      SubEventID : 0
  SubEventStream : 
]


In [28]:
print(frame_physics['I3EventHeader'])

[ I3EventHeader:
        StartTime: 1905-06-04 18:35:44.000,000,000,0 UTC
         EndTime : 1905-06-04 18:35:44.000,000,000,0 UTC
           RunID : 4910
        SubrunID : 4294967295
         EventID : 7
      SubEventID : 0
  SubEventStream : 
]


# 12- I3MCTree_RNGState

In [29]:
print(frame_daq['I3MCTree_RNGState'])

In [30]:
print(frame_physics['I3MCTree_RNGState'])

# 13- MMCTrackList

In [31]:
print(frame_daq['MMCTrackList'])

In [32]:
print(frame_physics['MMCTrackList'])

# 14- I3MCTree_postprop

In [33]:
print(frame_daq['I3MCTree_postprop'])

[I3MCTree:
  136 NuMu (-589.976m, 399.972m, 296.257m) (138.196deg, 321.854deg) 0ns 993.477GeV 0m Primary
    130 MuMinus (-589.976m, 399.972m, 296.257m) (136.385deg, 317.974deg) 0ns 179.12GeV 656.237m Dark
      140 MuMinus (-589.976m, 399.972m, 296.257m) (136.407deg, 318.018deg) 0ns 179.12GeV 14.4554m Null
      141 PairProd (-597.386m, 406.64m, 306.725m) (136.407deg, 318.018deg) 48.2182ns 0.50808GeV 14.4554m Null
      142 MuMinus (-597.386m, 406.64m, 306.725m) (136.406deg, 318.089deg) 48.2182ns 175.066GeV 13.1799m Null
      143 DeltaE (-604.147m, 412.713m, 316.271m) (136.406deg, 318.089deg) 92.1817ns 0.752999GeV 27.6354m Null
      144 MuMinus (-604.147m, 412.713m, 316.271m) (136.515deg, 318.141deg) 92.1817ns 171.082GeV 45.3299m Null
      145 DeltaE (-627.372m, 433.55m, 349.153m) (136.515deg, 318.141deg) 243.386ns 1.63372GeV 72.9653m Null
      146 MuMinus (-627.372m, 433.55m, 349.153m) (136.396deg, 318.133deg) 243.386ns 158.342GeV 54.1739m Null
      147 PairProd (-655.163m, 458.

In [34]:
print(frame_physics['I3MCTree_postprop'])

[I3MCTree:
  136 NuMu (-589.976m, 399.972m, 296.257m) (138.196deg, 321.854deg) 0ns 993.477GeV 0m Primary
    130 MuMinus (-589.976m, 399.972m, 296.257m) (136.385deg, 317.974deg) 0ns 179.12GeV 656.237m Dark
      140 MuMinus (-589.976m, 399.972m, 296.257m) (136.407deg, 318.018deg) 0ns 179.12GeV 14.4554m Null
      141 PairProd (-597.386m, 406.64m, 306.725m) (136.407deg, 318.018deg) 48.2182ns 0.50808GeV 14.4554m Null
      142 MuMinus (-597.386m, 406.64m, 306.725m) (136.406deg, 318.089deg) 48.2182ns 175.066GeV 13.1799m Null
      143 DeltaE (-604.147m, 412.713m, 316.271m) (136.406deg, 318.089deg) 92.1817ns 0.752999GeV 27.6354m Null
      144 MuMinus (-604.147m, 412.713m, 316.271m) (136.515deg, 318.141deg) 92.1817ns 171.082GeV 45.3299m Null
      145 DeltaE (-627.372m, 433.55m, 349.153m) (136.515deg, 318.141deg) 243.386ns 1.63372GeV 72.9653m Null
      146 MuMinus (-627.372m, 433.55m, 349.153m) (136.396deg, 318.133deg) 243.386ns 158.342GeV 54.1739m Null
      147 PairProd (-655.163m, 458.

# 15- TriggerTime_3PMT_2DOM

In [35]:
print(frame_daq['TriggerTime_3PMT_2DOM'])

I3Double(-1397)


In [36]:
print(frame_physics['TriggerTime_3PMT_2DOM'])

I3Double(-1397)


TriggerTime_3PMT_2DOM is created by DetectorTrigger 

$$
\text{mintrigtime} = \min\left(\text{detectorTriggerTime} \cup \text{stringTriggerTime} \cup \text{singleOMTriggerTime}\right)
$$


# 16- StringTriggers_3PMT_2DOM

In [37]:
print(frame_daq['StringTriggers_3PMT_2DOM'])

[]


In [38]:
print(frame_physics['StringTriggers_3PMT_2DOM'])

[]


StringTriggers_3PMT_2DOM is created by DetectorTrigger

###### check if this is true:

### String trigger time (`stringTriggerTime`) — how it is produced

- `stringTriggerTime` is a list of trigger timestamps only, with format:

  $$
  [t_1, t_2, t_3, \dots]
  $$

- A **string group** is an OM set defined by geometry: typically OMs on **neighboring string(s)** within a certain **vertical OM-row range** (`StringNRows`). If `ForceAdjacency=True`, only strings that are sufficiently close are included. Coincidence candidates are accumulated and counted **within these groups**.

#### Algorithm (high level)

1. **Per-OM coincidence candidates:**  
   For each OM, if at least `OMPMTCoinc` PMTs fire within the relevant coincidence time window, that OM contributes one or more **candidate times** (OM–time pairs).

2. **Group-level coincidence test:**  
   Candidate times are assigned to the corresponding **string group(s)** of the OM.  
   Within each group, if within `StringCoincidenceWindow` there are candidate times coming from **at least `StringCoincidenceN` distinct OMs**, the group forms a **string trigger**.

3. **Representative trigger time:**  
   For each triggered group, a single representative time is written to `stringTriggerTime`.  
   The representative time is chosen as the **latest time (max)** among the contributing candidate times in that group.

4. **Merging nearby triggers:**  
   If two trigger times in `stringTriggerTime` are very close, they are merged into one entry (keeping the representative value according to the implementation; in this description, the chosen time remains the group’s latest time).


# 17- singleOMTrigger_3PMT_2DOM

In [39]:
print(frame_physics['singleOMTrigger_3PMT_2DOM'])

[492, 117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153, 270, 271, 287, 291, -1397]


In [40]:
print(frame_daq['singleOMTrigger_3PMT_2DOM'])

[492, 117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153, 270, 271, 287, 291, -1397]


`singleOMTriggerTime` is generated by `DetectorTrigger`.

###### check if this is true:
#### How it is produced

From `DOMTrigger_Time/NCoin`, timestamps are selected where, **within a single OM**, the coincidence count is **≥ 3**.  
These selected times are appended to the `singleOMTriggerTime` list.


# 18- DetectorTriggers_3PMT_2DOM

In [41]:
print(frame_physics['DetectorTriggers_3PMT_2DOM'])

[]


In [42]:
print(frame_daq['DetectorTriggers_3PMT_2DOM'])

[]


`DetectorTriggers_3PMT_2DOM`is `DetectorTrigger`

##### check if this is true

#### How it is produced (high level)

1. **Build OM–time candidates:**  
   Start with an empty list. For each OM, if the number of coincident PMT hits within that OM is at least `OMPMTCoinc` (often ≥ 2), add an **(OM ID, time)** candidate to the list.

2. **Full-detector coincidence test:**  
   For each candidate time, treat it as a reference and look within a global time window of width `FullDetectorCoincidenceWindow`.  
   If, within that window, the number of **distinct OMs** contributing candidates is at least `FullDetectorCoincidenceN` (e.g., ≥ 3 OMs), then a detector trigger is formed.

3. **Representative trigger time:**  
   When a detector trigger is formed, the time written to `detectorTriggerTime` is the **latest time (max)** among the contributing candidates in that coincidence window.

4. **Merge nearby triggers:**  
   Trigger times that occur close to each other are merged within `EventLength` (so very near triggers become a single entry).


# 19- DOMTrigger_HitTimes

`DOMTrigger_HitTimes` is produced by the **DOM Trigger** logic (not by the **Detector Trigger**).

In [43]:
print(frame_daq['DOMTrigger_HitTimes'])

[OMKey(264,17,0) => [484.234, 492.897, 490.501],
OMKey(265,16,0) => [117.084, 116.052, 117.269, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 121.025, 117.084, 118.571, 119.463, 118.012, 127.617, 121.025, 118.571, 127.617, 119.463, 128.694, 121.025, 119.463, 129.301, 127.617, 121.025, 128.694, 129.301, 137.501, 127.617, 128.694, 137.253, 137.501, 137.253, 129.301, 139.191, 137.501, 137.253, 140.215, 139.191, 137.501, 144.596, 140.215, 137.253, 139.191, 137.501, 144.596, 140.215, 145.116, 137.253, 139.191, 149.33, 144.596, 140.215, 145.116, 149.33, 144.596, 150.827, 145.116, 150.827, 149.33, 144.596, 152.015, 152.496, 145.116, 150.827, 149.33, 144.596, 153.458, 152.015, 152.496, 145.116],
OMKey(265,17,0) => [270.082, 270.955, 269.831, 270.346, 270.955, 270.082, 269.831, 271.297, 270.346, 271.544, 282.714, 287.267, 284.435, 282.714, 291.178, 287.267, 284.435],
OMKey(330,6,0) => [-1398.87, -1398.46, -

In [44]:
print(frame_physics['DOMTrigger_HitTimes'])

[OMKey(264,17,0) => [484.234, 492.897, 490.501],
OMKey(265,16,0) => [117.084, 116.052, 117.269, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 121.025, 117.084, 118.571, 119.463, 118.012, 127.617, 121.025, 118.571, 127.617, 119.463, 128.694, 121.025, 119.463, 129.301, 127.617, 121.025, 128.694, 129.301, 137.501, 127.617, 128.694, 137.253, 137.501, 137.253, 129.301, 139.191, 137.501, 137.253, 140.215, 139.191, 137.501, 144.596, 140.215, 137.253, 139.191, 137.501, 144.596, 140.215, 145.116, 137.253, 139.191, 149.33, 144.596, 140.215, 145.116, 149.33, 144.596, 150.827, 145.116, 150.827, 149.33, 144.596, 152.015, 152.496, 145.116, 150.827, 149.33, 144.596, 153.458, 152.015, 152.496, 145.116],
OMKey(265,17,0) => [270.082, 270.955, 269.831, 270.346, 270.955, 270.082, 269.831, 271.297, 270.346, 271.544, 282.714, 287.267, 284.435, 282.714, 291.178, 287.267, 284.435],
OMKey(330,6,0) => [-1398.87, -1398.46, -

# 20- DOMTrigger_NCoin

`DOMTrigger_NCoin` is produced by the **DOM Trigger** logic (not by the **Detector Trigger**).


In [45]:
print(frame_daq['DOMTrigger_NCoin'])

[OMKey(264,17,0) => [3],
OMKey(265,16,0) => [3, 5, 6, 7, 5, 4, 5, 5, 4, 4, 5, 6, 4, 4, 6, 7],
OMKey(265,17,0) => [4, 6, 3, 4],
OMKey(330,6,0) => [3]]


In [46]:
print(frame_physics['DOMTrigger_NCoin'])

[OMKey(264,17,0) => [3],
OMKey(265,16,0) => [3, 5, 6, 7, 5, 4, 5, 5, 4, 4, 5, 6, 4, 4, 6, 7],
OMKey(265,17,0) => [4, 6, 3, 4],
OMKey(330,6,0) => [3]]


# 21- DOMTrigger_Time

`DOMTrigger_Time` is produced by the **DOM Trigger** logic (not by the **Detector Trigger**).

In [47]:
print(frame_daq['DOMTrigger_Time'])

[OMKey(264,17,0) => [492],
OMKey(265,16,0) => [117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153],
OMKey(265,17,0) => [270, 271, 287, 291],
OMKey(330,6,0) => [-1397]]


In [48]:
print(frame_physics['DOMTrigger_Time'])

[OMKey(264,17,0) => [492],
OMKey(265,16,0) => [117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153],
OMKey(265,17,0) => [270, 271, 287, 291],
OMKey(330,6,0) => [-1397]]


# 22- DOMTrigger_PMTs

`DOMTrigger_PMTs` is produced by the **DOM Trigger** logic (not by the **Detector Trigger**).

In [49]:
print(frame_daq['DOMTrigger_PMTs'])

[OMKey(264,17,0) => [13, 12, 16],
OMKey(265,16,0) => [11, 7, 15, 7, 15, 2, 11, 16, 3, 7, 15, 2, 11, 16, 3, 7, 15, 2, 8, 11, 16, 3, 2, 11, 8, 16, 11, 3, 7, 8, 3, 16, 11, 8, 7, 16, 8, 11, 7, 15, 8, 15, 16, 7, 8, 15, 3, 7, 8, 4, 3, 15, 7, 8, 4, 3, 6, 15, 7, 15, 4, 3, 6, 15, 4, 16, 6, 16, 15, 4, 3, 11, 6, 16, 15, 4, 7, 3, 11, 6],
OMKey(265,17,0) => [9, 12, 11, 10, 12, 9, 11, 13, 10, 15, 11, 10, 3, 11, 12, 10, 3],
OMKey(330,6,0) => [14, 9, 12]]


In [50]:
print(frame_physics['DOMTrigger_PMTs'])

[OMKey(264,17,0) => [13, 12, 16],
OMKey(265,16,0) => [11, 7, 15, 7, 15, 2, 11, 16, 3, 7, 15, 2, 11, 16, 3, 7, 15, 2, 8, 11, 16, 3, 2, 11, 8, 16, 11, 3, 7, 8, 3, 16, 11, 8, 7, 16, 8, 11, 7, 15, 8, 15, 16, 7, 8, 15, 3, 7, 8, 4, 3, 15, 7, 8, 4, 3, 6, 15, 7, 15, 4, 3, 6, 15, 4, 16, 6, 16, 15, 4, 3, 11, 6, 16, 15, 4, 7, 3, 11, 6],
OMKey(265,17,0) => [9, 12, 11, 10, 12, 9, 11, 13, 10, 15, 11, 10, 3, 11, 12, 10, 3],
OMKey(330,6,0) => [14, 9, 12]]


# Comparison Between EventPulseSeries and EventPulseSeries_nonoise

In [51]:
import bisect


def time_in_sorted_list(t, sorted_list, tol=1e-3):
    """t is in sorted_list within +/- tol? (sorted_list must be sorted)"""
    i = bisect.bisect_left(sorted_list, t)
    if i < len(sorted_list) and abs(sorted_list[i] - t) <= tol:
        return True
    if i > 0 and abs(sorted_list[i-1] - t) <= tol:
        return True
    return False

tol = 0  

missing = []  
stats = {
    "keys_in_nonoise": 0,
    "keys_missing_in_noisy": 0,
    "nonoise_pulses_total": 0,
    "matched_total": 0,
    "missing_total": 0,
}

for k in EventPulseSeries_nonoise_physics:
    stats["keys_in_nonoise"] += 1

    if k not in EventPulseSeries_physics:
        stats["keys_missing_in_noisy"] += 1
        for p in EventPulseSeries_nonoise_physics[k]:
            stats["nonoise_pulses_total"] += 1
            stats["missing_total"] += 1
            missing.append((k, p.time))
        continue

    noisy_times = sorted(p.time for p in EventPulseSeries_physics[k])

    for p in EventPulseSeries_nonoise_physics[k]:
        stats["nonoise_pulses_total"] += 1
        if time_in_sorted_list(p.time, noisy_times, tol=tol):
            stats["matched_total"] += 1
        else:
            stats["missing_total"] += 1
            missing.append((k, p.time))

print("=== Summary ===")
print("keys_in_nonoise:", stats["keys_in_nonoise"])
print("keys_missing_in_noisy:", stats["keys_missing_in_noisy"])
print("nonoise_pulses_total:", stats["nonoise_pulses_total"])
print("matched_total:", stats["matched_total"])
print("missing_total:", stats["missing_total"])
print("match_fraction:", stats["matched_total"] / max(1, stats["nonoise_pulses_total"]))




=== Summary ===
keys_in_nonoise: 26
keys_missing_in_noisy: 0
nonoise_pulses_total: 46
matched_total: 46
missing_total: 0
match_fraction: 1.0


In [52]:
missing

[]

In [53]:

n_keys_noisy   = len(EventPulseSeries_physics)
n_pulses_noisy = sum(len(EventPulseSeries_physics[k]) for k in EventPulseSeries_physics)

n_keys_nonoise   = len(EventPulseSeries_nonoise_physics)
n_pulses_nonoise = sum(len(EventPulseSeries_nonoise_physics[k]) for k in EventPulseSeries_nonoise_physics)

print("Noisy   : keys =", n_keys_noisy,   "| pulses =", n_pulses_noisy)
print("NoNoise : keys =", n_keys_nonoise, "| pulses =", n_pulses_nonoise)
print("Extra (Noisy - NoNoise): keys =", n_keys_noisy - n_keys_nonoise,
      "| pulses =", n_pulses_noisy - n_pulses_nonoise)


Noisy   : keys = 3733 | pulses = 3881
NoNoise : keys = 26 | pulses = 46
Extra (Noisy - NoNoise): keys = 3707 | pulses = 3835


# Pulse Comparison

In [54]:
pulses_Accepted_PulseMap_physics = Accepted_PulseMap_physics[key1]
print(pulses_Accepted_PulseMap_physics)

[[I3RecoPulse:
             Time : 358.755
           Charge : 1
            Width : nan
            Flags : 
], [I3RecoPulse:
             Time : 356.031
           Charge : 1
            Width : nan
            Flags : 
]]


In [55]:
pulses_PMT_Response_nonoise_physics = PMT_Response_nonoise_physics[key1]
print(pulses_PMT_Response_nonoise_physics)

[[I3RecoPulse:
             Time : 353.276
           Charge : 0.770366
            Width : 10
            Flags : 
]]


In [56]:
pulses_PMT_Response_physics = PMT_Response_physics[key1]
print(pulses_PMT_Response_physics)

[[I3RecoPulse:
             Time : 353.276
           Charge : 1.35644
            Width : 10
            Flags : 
]]


In [57]:
pulses_EventPulseSeries_physics = EventPulseSeries_physics[key1]
print(pulses_EventPulseSeries_physics)

[[I3RecoPulse:
             Time : 3750.28
           Charge : 1.35644
            Width : nan
            Flags : 
]]


In [58]:
pulses_EventPulseSeries_nonoise_physics = EventPulseSeries_nonoise_physics[key1]
print(pulses_EventPulseSeries_nonoise_physics)

[[I3RecoPulse:
             Time : 3750.28
           Charge : 0.770366
            Width : nan
            Flags : 
]]


In [59]:
pulses_Noise_K40_physics = Noise_K40_physics[key3]
print(pulses_Noise_K40_physics)

[[I3RecoPulse:
             Time : -990.217
           Charge : 1
            Width : 9
            Flags : 
]]


In [60]:
pulses_Noise_Dark_physics = Noise_Dark_physics[key2]
print(pulses_Noise_Dark_physics)

[[I3RecoPulse:
             Time : 5541.05
           Charge : 1
            Width : 10
            Flags : 
]]


triggerpulsemap is OM level. it includes real pulse and noise

In [61]:
pulses_triggerpulsemap_physics = triggerpulsemap_physics[key4]  ## key4 is OM level
print(pulses_triggerpulsemap_physics)

[[I3RecoPulse:
             Time : 353.276
           Charge : 1.35644
            Width : 10
            Flags : 
], [I3RecoPulse:
             Time : 2413.94
           Charge : 0.559609
            Width : 4
            Flags : 
], [I3RecoPulse:
             Time : 4776.49
           Charge : 1.15494
            Width : 3
            Flags : 
]]


In [62]:
pulses_triggerpulsemap_daq = triggerpulsemap_daq[key4]  ## key4 is OM level
print(pulses_triggerpulsemap_daq)

[[I3RecoPulse:
             Time : 353.276
           Charge : 1.35644
            Width : 10
            Flags : 
], [I3RecoPulse:
             Time : 2413.94
           Charge : 0.559609
            Width : 4
            Flags : 
], [I3RecoPulse:
             Time : 4776.49
           Charge : 1.15494
            Width : 3
            Flags : 
]]


## Realizations & Questions

### Observations
- `PMT_Response_nonoise` includes **pulse width**, while `Accepted_PulseMap` does **not** include width information.
- In `Accepted_PulseMap`, the **charge is always 1**.
- The **width information appears to be lost** when converting from `EventPulseSeries` to `Accepted_PulseMap`.

### Open question
- Why is the **time in `Accepted_PulseMap` later** than the corresponding time in `PMT_Response` / `PMT_Response_nonoise`?
